<a href="https://colab.research.google.com/github/adalbertii/Modele-klasyfikacyjne/blob/main/01_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Model klasyfikacji binarnej**
```
!pip install scikit-learn
```

### <a name='a1'></a> Import bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import seaborn as sns
import sklearn

sns.set(font_scale=1.3)
np.set_printoptions(precision=6, suppress=True, edgeitems=10, linewidth=100000,
                    formatter=dict(float=lambda x: f'{x:.2f}'))
np.random.seed(42)
sklearn.__version__

###  <a name='a2'></a>  Regresja Logistyczna (Logistic Regression) -

Pomimo nazwy jest to liniowy model do zadań klasyfikacyjnych. Inna nazwa Logit Regression.

#### Przykłady zastosowań:
- przewidywanie czy mail jest spamem, czy też nie
- przewidywanie czy użytkownik kliknie w reklamę
- przewidywanie czy nowotwór jest złośliwy czy też nie
- przewidywanie czy dłużnik spłaci wierzycielowi dług, czy też zajdzie zdarzenie default
- przewidywanie czy transakcja jest próbą oszustwa

Przy budowie modelu regresji logistycznej wykorzystamy funkcję sigmoid. Definiuje się ją wzorem:
$$sigmoid(x) = \frac{1}{1+e^{-x}}$$

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

X = np.arange(-5, 5, 0.1)
y = sigmoid(X)

plt.figure(figsize=(8, 6))
plt.plot(X, y)
plt.title('Funkcja Sigmoid')
plt.show()

Następnie rozważmy funkcję liniową $y = w_0 + w_1x$. Podstawiając to do funkcji sigmoid otrzymujemy:
$$p(x) = \frac{1}{1 + e^{-(w_0 + w_1x)}}$$
Dzięki temu przekształceniu regresja logistyczna zwraca nam wartości z przedziału $(0, 1)$ co możemy interpretować jako prawdopodobieństwo i na podstawie tych prawdopodobieństw przewidywać poszczególne klasy.

###  <a name='a3'></a> Załadowanie danych

In [ ]:
from sklearn.datasets import load_breast_cancer

raw_data = load_breast_cancer()
raw_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [ ]:
print(raw_data.DESCR)

In [ ]:
print(raw_data.feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area' 'mean smoothness' 'mean compactness' 'mean concavity' 'mean concave points' 'mean symmetry' 'mean fractal dimension' 'radius error' 'texture error' 'perimeter error' 'area error' 'smoothness error' 'compactness error' 'concavity error' 'concave points error' 'symmetry error' 'fractal dimension error' 'worst radius' 'worst texture' 'worst perimeter' 'worst area' 'worst smoothness' 'worst compactness' 'worst concavity' 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [ ]:
all_data = raw_data.copy()

data = all_data['data']
target = all_data['target']

print(f'rozmiar data: {data.shape}')
print(f'rozmiar target: {target.shape}')

In [ ]:
# tworzę dataframe na potrzeby wizualizacji i wtępnej analizy danych
df_a = pd.DataFrame(data,columns=raw_data.feature_names)
#df_a.columns = 'Imię', 'Wiek'
df_a.head()

In [ ]:
# do wczesniej zdefiniowanego  dataframe dodaje kolumne ze zmienna objasnianą
df_a['outcome'] = pd.DataFrame(target)

In [ ]:
df_a.head()

In [ ]:
# analiza statystyczna zbioru danych
df_a.describe().T

In [ ]:
# sprawdzamy korelacje zmiennych
df_a.corr()['outcome'].sort_values(ascending=False)

In [ ]:
#sprawdzenie rozkładu unikalnych wartości zmiennej objaśnianej
print(np.unique(df_a['outcome'], return_counts=True))

###  <a name='a4'></a> Podział danych na zbiór treningowy i testowy

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

###  <a name='a44'></a> Przygotowanie danych do modelu

In [ ]:
X_train[1]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# ważne !!!!
# tylko na danych treningowych
scaler.fit(X_train)

# skalujemy zarówno dane treningowe jak i testowe
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train[1]

In [ ]:
scaler.mean_

In [ ]:
# odchylenie standardowe
scaler.scale_

###<a name='a5'></a>  Dopasowanie modelu

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

###<a name='a6'></a>  Predykcja na podstawie modelu

In [ ]:
y_pred = log_reg.predict(X_test)
y_pred[:30]

In [ ]:
# wartości prawdopodobieństw dla klasy 0 i dla klasy 1
y_prob = log_reg.predict_proba(X_test)
y_prob[:5]

###<a name='a7'></a>  Ocena modelu

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from mlxtend.plotting import plot_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
#plot_confusion_matrix(cm)
print(cm)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
def plot_confusion_matrix(cm):
    # klasyfikacja binarna
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index),
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))



**Precision** - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne

**Recall** - jak wiele obserwacji z wszystkich poytywnych sklasyfikowaliśmy jako pozytywne